## Введение в анализ данных и машинное обучение на Python

### Тема 11: Анализ набора данных «Титаник»

---

[Данные](http://www.machinelearning.ru/wiki/images/3/32/Titanic.zip)

[Описание данных](https://www.kaggle.com/c/titanic/data)

[Хороший пример](https://towardsdatascience.com/predicting-the-survival-of-titanic-passengers-30870ccc7e8), но на английском.

#### Постановка задачи регрессии

**Исследовательский вопрос:**

**Гипотеза:**

**Механизм:**

**Признаки:**

**Методы:**


#### Постановка задачи классификации

**Исследовательский вопрос:**

**Гипотеза:**

**Механизм:**

**Признаки:**

**Методы:**

#### Выводы